# Reconstruccion de Imagenes Submarinas

Este cuaderno se usa para comprobar la calidad de la reconstruccion de la red tanto en imagenes como en video

**Librerias**

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from IPython.display import clear_output
from time import time
%tensorflow_version 2.x

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **MODELO Y FUNCIONES**

In [4]:
recons = tf.keras.models.load_model('/content/drive/My Drive/DeepLearning/ResultadosFinales/Modelos/SRreconsGAN.h5', compile = False)

In [5]:
def resize(inimg, heigth, width):
  inimg = tf.image.resize(inimg, [heigth, width])   # se transforma la imagen de entrada (la que se ve mal)
  return inimg

In [6]:
def modifyFrame(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  imagenp = np.asarray(image, dtype=np.float32)
  imagenp = resize(imagenp,256,256)
  imagenp = (imagenp / 127.5) - 1 
  imagenp = imagenp[np.newaxis,:,:,:]
  return imagenp

In [7]:
def remodifyFrame(image):
  imagenp = image[0,:,:,:]
  imagenp = (imagenp + 1) * 127.5
  imagenp = np.asarray(imagenp, dtype=np.uint8)
  imagenp = cv2.cvtColor(imagenp, cv2.COLOR_RGB2BGR)
  return imagenp

In [8]:
def reconsSR_image(image):
  modFrame = modifyFrame(image)
  reconsFrame = resize(modFrame,1024,1024)
  srFrame = recons(reconsFrame)
  frame = remodifyFrame(srFrame)
  return frame

# **TIEMPO REAL (VIDEO)**

In [9]:
!nvidia-smi

Mon Oct  5 18:47:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    29W /  70W |    739MiB / 15079MiB |      8%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
cap = cv2.VideoCapture('/content/drive/My Drive/DeepLearning/ResultadosFinales/FramesInput.avi')

#width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#out = cv2.VideoWriter('/content/drive/My Drive/DeepLearning/ResultadosFinales/output.avi', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (width, height))
i = 0
sumtime = 0.0

f = open("/content/drive/My Drive/DeepLearning/ResultadosFinales/time_process.log","w")
while True:
    res, video = cap.read()

    if res == True:
      start_time = time()
      frame = reconsSR_image(video)
      frame_time = time() - start_time
      f.write("Frame "+str(i)+" - "+str(frame_time)+"\n")
      i = i+1
      sumtime = sumtime + frame_time
      #cv2_imshow(frame)

      if cv2.waitKey(1) & 0xFF == ord('x'):
        break
    else:
      break

f.write("Tiempo medio = "+str(sumtime/i)+"\n")
f.write("FPS medio= "+str(i/sumtime)+"\n")
cap.release()
cv2.destroyAllWindows()
f.close()


# **Reconstruccion de imagenes**


In [ ]:
i = cv2.imread('/content/drive/My Drive/DeepLearning/recons+sr/Input/Input/569.jpg')
cv2_imshow(i)

In [ ]:
img = reconsSR_image(i)
cv2_imshow(img)